<a href="https://colab.research.google.com/github/MUMADE-TADM/S1-python-y-S5-Titanic-/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Yahoo finance](https://finance.yahoo.com/)

In [ ]:
!pip install -U yfinance pandas_datareader

# Datos:

Valor de cierre del IBEX.

In [ ]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
yf.pdr_override() # <== that's all it takes :-)
data = pdr.get_data_yahoo("^IBEX", start="2020-01-01", end="2020-10-30")

## [Bollinger Bands](https://en.wikipedia.org/wiki/Bollinger_Bands):

El 97% de los valores de un activo se encuentran entre un máximo de $+1.96 \times  \sigma(20\ valores\ anteriores)$ y $-1.96 \times \sigma(20\ valores\  anteriores)$ centrados en la media de los 20 valores anteriores.

(Las bandas las calcula con los 20 valores anteriores). 

In [ ]:
d=pd.DataFrame()
d['R']=data['Close'] #VALORES DE CIERRE. LOS LLAMAMOS REAL(R).
d['M']=data['Close'].rolling(20).mean() #HACEMOS LA MEDIA DE CADA 20 VALORES. 
d['+M']=d['M']+data['Close'].rolling(20).std()*1.96
d['-M']=d['M']-data['Close'].rolling(20).std()*1.96
d.plot(figsize=(15,5))

## Transformación de los datos para un problema de regresión / clasificación-
- Datos originales $[c_0,\ldots,c_T]$
- Datos transformados:$[[c_0,\ldots,c_{19}],c_{20}],\ldots,[[c_{T-20},\ldots,c_{T-1}],c_T]$.

Cada dato está compuesto por dos partes (una independiente y otra dependiente).

In [ ]:
def windowData(s,window_input=1,window_output=1,step=1):
  X=[] 
  Y=[] #la que quiero obtener
  #Datos necesarios
  dn=window_input+window_output
  #Calcular cuantos pasos completos podemos realizar. Los pasos son el tamaño de la ventana. Dividimos en tantos pasos como tengamos 
  #sin contar el último bloque [input,output]
  ld=int((len(s)-dn)/step)*step
  for i in range(len(s)-ld-1,len(s)-dn,step): # bucle desde el primer elemento posible hasta el máximo que podamos. 
    X.append(s[i:i+window_input]) #desde donde toque hasta el tamaño del input. 
    Y.append(s[i+window_input:i+dn]) 
  return X,Y

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#Escalanos los datos
dScale = MinMaxScaler()
ldatos=d['R'].values #ldatos.shape=(len(d['R']),)
ldatos=ldatos.reshape((len(d['R']),1)) #ldatos.shape=(len(d['R']),1). Ponemos que cada elemento tenga su propia dimensión, por eso ponemos ,1.
dScale.fit(ldatos)
ldatos=dScale.transform(ldatos).reshape((len(ldatos),))

#Contruimos los conjunto de datos X e Y
X,Y=windowData(ldatos,window_input=20,window_output=1,step=1) 
X=np.array(X)
Y=np.array(Y)
#Lo mezclo para obtener el conjunto de datos de test y train. 
#partmos los conjuntos en entranamiento y test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.33,shuffle=False, random_state=42)

Ahora ya tenemos nuestros datos preparados para la red neuronal(Perceptron).

# PERCEPTRON:

## Librerías necesarias:
- [Keras](https://keras.io/)
- [Tensorflow](https://www.tensorflow.org/)

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential #me permite colocar la salida de una red en la entrada de la siguiente.
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras import metrics

## Definición de la Red:

Definimos nuestro Perceptron con las siguientes capas (**layers**):
- Modelo secuencial [Sequential](https://keras.io/api/models/sequential/).
- Capa de entrada: [Input](https://keras.io/api/layers/core_layers/input/).
- Capa Densa: [Dense](https://keras.io/api/layers/core_layers/dense/). 

In [ ]:
perceptron=Sequential() 
perceptron.add(Input(20)) #input: cada uno de los datos de entrada que teníamos. CAPA 1.
perceptron.add(Dense(30,activation='relu')) #La salida de la capa de la línea de arriba va a ir a una capa densa. Quiero 30 neuronas en esta capa densa y además quiero que la función de activación de las neuronas de esta capa tenga una activación relu. CAPA 2. 
perceptron.add(Dense(30,activation='relu')) #La salida de la línea anterior va a otra capa densa. CAPA 3.  
perceptron.add(Dense(1, activation='sigmoid')) #CAPA 4. Una sóla neurona y con valores de 0 ó 1 (sigmoid).
perceptron.summary() #Arquitectura resumen. 

Los nº de Param que vemos en el resultado son los w.


## Compilar la red:
Definido el **Perceptrón**, lo compilamos indicando:
- Que optimizador utilizar (gradiente).
- Que función de error (pérdida o **loss**) (ej. distancia cuadrática, entropía...).
- Que métricas observar cuando lo entrenemos .

Función [**compile**](https://keras.io/api/models/model_training_apis/#compile-method)

Esto nos permite hacer los tensores.

In [ ]:
# Compilamos el modelo:
perceptron.compile(
    optimizer='adam', #adam es un algoritmo de cálculo de gradientes. 
    loss='mean_squared_error', #La función en este caso es el error cuadrático.
    metrics=[
        metrics.MeanSquaredError(name='my_mse'),
        metrics.AUC(name='my_auc'),
    ]
)

## Aprender los parámetros:

Realizamos el **aprendizaje** indicando:
- Los conjuntos de datos a utilizar **X** e **Y**.
- Que proporción de los datos utilizaremos para validar el modelo.
- El tamaño del conjunto **batch**.
- El número de **épocas** a realizar.

Método [**fit**](https://keras.io/api/models/model_training_apis/#fit-method).

In [ ]:
history=perceptron.fit(X,Y,validation_split=0.33, batch_size=10 ,epochs=300,verbose=0) 
#batch_size:tamaño, y epochs: nº de veces que pasa por los datos. 

In [ ]:
history.history.keys() 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch') 
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Lo ideal es que train y test sean muy parecidos (se ajusten bien).
Para generalizar mucho normalmente penalizo este ajuste.

#Construimos el predictor:

## Utilizar la red:
Utilizar el perceptron aprendido con el método [**predict**](https://keras.io/api/models/model_training_apis/#predict-method).

In [ ]:
Y0=perceptron.predict(X_train) 
Y1=perceptron.predict(X_test)

In [ ]:
r=pd.DataFrame(np.vstack((Y_train,Y_test)),columns=['R'])
r['Perceptron']=pd.DataFrame(np.vstack((Y0,Y1)))
r.plot(figsize=(15,5))
#Pintamos lo que da perceptron respecto al valor real. Son las predicciones sobre el conjunto de test.

Se ajusta bastante bien, pero suaviza los picos y tiende a infravalorar. Además, a partir del 100 tiende a adelantarse un poco al valor real. 

Lo que podemos hacer para solucionar estas pequeñas cosas, es coger ventanas más pequeñas. 

## Salvar y Cargar Redes:
- Salvar mediante el método [**save**](https://keras.io/api/models/model_saving_apis/#save-method).
- Recuperar un modelo con el método [**load_model**](keras.models.load_model).

(Guardo mi modelo).

In [ ]:
perceptron.save('perceptron.h5')
otro_perceptron=keras.models.load_model('perceptron.h5')
#LOS FICHEROS H5 SON FICHEROS JERÁRQUICOS. ESTÁN DISEÑADOS PARA ALMACENAR TENSORES DE FORMA JERÁRQUICA. 